In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
# Load the data
df_idgs = pd.read_excel('/Users/rudyhendrawan/miniforge3/datasets/DTPR-UPDATE-3.xlsx')
df_samples = df_idgs.sample(3)
df_samples

,NO,NIDN,NAMA,NAMA DOSEN DENGAN GELAR,JAFUNG,SERDOS,scholar_id
38,39,806079201.0,I WAYAN JEPRIANA,"I WAYAN JEPRIANA, S.Kom., M.Cs",Asisten Ahli,2024.0,qjU7YEgAAAAJ
37,38,802098601.0,I WAYAN GEDE NARAYANA,"I WAYAN GEDE NARAYANA, S.Kom., M.Kom",Lektor 200,2018.0,GqPl4-4AAAAJ
34,35,813049001.0,I PUTU GEDE ABDI SUDIATMIKA,"I PUTU GEDE ABDI SUDIATMIKA, S.Pd., M.Kom",Lektor 300,2020.0,Ztn650AAAAAJ


In [3]:
# Function to get publication data
def get_publication_data(scholar_id, author_name, delay_times=1.0):
    url = f'https://scholar.google.com/citations?user={scholar_id}&hl=en'
    browser.get(url)
    logging.info(f'Getting publication data for scholar_id: {scholar_id} - {author_name}')
    time.sleep(delay_times)

    # Click the "Show more" button
    while True:
        try:
            show_more_button = browser.find_element('id', 'gsc_bpf')
            if show_more_button.is_displayed():
                show_more_button.click()
                logging.info('Clicking the "Show more" button')
                time.sleep(delay_times)
                break
            else:
                logging.info('No more "Show more" button')
                break
        except Exception as e:
            break

    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

    publications = []
    for row in soup.find_all('tr', class_='gsc_a_tr'):
        title_elem = row.find('a', class_='gsc_a_at')
        title = row.find('a', class_='gsc_a_at').text
        year = row.find('span', class_='gsc_a_hc').text
        citations = row.find('a', class_='gsc_a_ac').text
        authors_publisher = row.find_all('div', class_='gs_gray')
        authors = authors_publisher[0].text
        publisher = authors_publisher[1].text

        article_url = title_elem['href'] if title_elem else 'N/A'
        article_id = article_url.split('citation_for_view=')[1] if article_url != 'N/A' else 'N/A'

        publications.append({
            'scholar_id': scholar_id,
            'title': title,
            'authors': authors,
            'publisher': publisher,
            'year': year,
            'citations': citations,
            'article_url': article_url,
            'article_id': article_id,
        })
        logging.info(f'Append publication data: {publications[-1]["title"]}')

    return publications

# Not stable yet, because the we need a scheduling mechanism to avoid the blocking of the data.
# Commented the function below.
# def fetch_publication_data(scholar_ids):
#     publications = []
#     with ThreadPoolExecutor(max_workers=5) as executor:
#         futures = [executor.submit(get_publication_data, scholar_id) for scholar_id in scholar_ids]
#         for future in as_completed(futures):
#             publications.extend(future.result())
#     return publications

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--no-sandbox')

# Initialize the WebDriver
browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# DataFrame to store the results
columns = ['scholar_id', 'title', 'authors', 'publisher', 'year', 'citations', 'article_url', 'article_id']
results_df = pd.DataFrame(columns=columns)

# Loop through each scholar_id and get the publication data
# Change the `df_idgs` to `df_samples` to test the function for fewer samples
for index, row in df_samples.iterrows():
    scholar_id = row['scholar_id']
    author_name = row['NAMA']
    publications = get_publication_data(scholar_id, author_name)
    results_df = pd.concat([results_df, pd.DataFrame(publications)], ignore_index=True)
    logging.info(f'Index-{index}: Finished getting publication data for scholar_id: {scholar_id} - {author_name}')

# Uncomment the below code if you commented the `fetch_publication_data` function
# scholar_ids = df_samples['scholar_id'].tolist()
# publications = fetch_publication_data(scholar_ids)
# df_publications = pd.DataFrame(publications)

# Save the results to a CSV file
# df_publications.to_csv('/Users/rudyhendrawan/miniforge3/datasets/publications.csv', index=False)

# Close the browser
browser.quit()

# Display the results
results_df.head()
# results_df.to_csv('/Users/rudyhendrawan/miniforge3/datasets/publications.csv', index=False)

2025-01-21 15:32:48,816 - INFO - ====== WebDriver manager ======
2025-01-21 15:32:48,977 - INFO - Get LATEST chromedriver version for google-chrome
2025-01-21 15:32:49,826 - INFO - Get LATEST chromedriver version for google-chrome
2025-01-21 15:32:50,538 - INFO - Driver [/Users/rudyhendrawan/.wdm/drivers/chromedriver/mac64/131.0.6778.264/chromedriver-mac-arm64/chromedriver] found in cache
2025-01-21 15:32:54,727 - INFO - Getting publication data for scholar_id: qjU7YEgAAAAJ - I WAYAN JEPRIANA
2025-01-21 15:32:55,842 - INFO - Clicking the "Show more" button
2025-01-21 15:32:56,862 - INFO - Append publication data: Konsep Algoritme dan Aplikasinya dalam Bahasa Pemrograman C++
2025-01-21 15:32:56,863 - INFO - Append publication data: Framework Itil V3 Domain Service Operation Dalam Analisis Pengelolaan Teknologi Blended Learning
2025-01-21 15:32:56,863 - INFO - Append publication data: Pelatihan Microsoft Office dan Desain Grafis untuk Perangkat Desa
2025-01-21 15:32:56,864 - INFO - Appen

,scholar_id,title,authors,publisher,year,citations,article_url,article_id
0,qjU7YEgAAAAJ,Konsep Algoritme dan Aplikasinya dalam Bahasa ...,"S Hanief, IW Jepriana","Penerbit Andi, 2020",2020,61,/citations?view_op=view_citation&hl=en&user=qj...,qjU7YEgAAAAJ:ufrVoPGSRksC
1,qjU7YEgAAAAJ,Framework Itil V3 Domain Service Operation Dal...,"S Hanief, IW Jefriana","Jurnal Teknologi Informasi dan Komputer 4 (1),...",2018,47,/citations?view_op=view_citation&hl=en&user=qj...,qjU7YEgAAAAJ:ULOm3_A8WrAC
2,qjU7YEgAAAAJ,Pelatihan Microsoft Office dan Desain Grafis u...,"DP Agustino, IW Jepriana, IPFE Putra","WIDYABHAKTI Jurnal Ilmiah Populer 3 (1), 38-48...",2020,35,/citations?view_op=view_citation&hl=en&user=qj...,qjU7YEgAAAAJ:_FxGoFyzp5QC
3,qjU7YEgAAAAJ,Analisis dan Implementasi Metode Item-based Co...,"IW Jepriana, S Hanief",Jurnal Nasional Pendidikan Teknik Informatika:...,2020,20,/citations?view_op=view_citation&hl=en&user=qj...,qjU7YEgAAAAJ:WF5omc3nYNoC
4,qjU7YEgAAAAJ,Analisis Maturity Level Dalam Blended Learning...,"S Hanief, IW Jepriana","Jurnal Teknologi Informasi Dan Komputer 5 (1),...",2019,13,/citations?view_op=view_citation&hl=en&user=qj...,qjU7YEgAAAAJ:roLk4NBRz8UC


In [4]:
def get_citation_data(article_id, article_url, delay_times=2.0):
	url = f'https://scholar.google.com{article_url}'
	try:
		browser.get(url)
		time.sleep(delay_times)  # Wait for the page to load
		logging.info(f'Getting citation data for article ID {article_id} - URL: {url}')
		html = browser.page_source
		soup = BeautifulSoup(html, 'html.parser')

		citations_per_year = {}

		# Find the graph wrapper
		graph_bars = soup.find('div', id='gsc_oci_graph_bars')
		if not graph_bars:
			print(f"No graph bars found for {url}")
			return citations_per_year

		# Extract year labels and citation counts
		years = graph_bars.find_all('span', class_='gsc_oci_g_t')
		citation_bars = graph_bars.find_all('a', class_='gsc_oci_g_a')

		# Ensure consistent pairing
		for year, bar in zip(years, citation_bars):
			year_value = year.text.strip()
			citation_count = bar.find('span', class_='gsc_oci_g_al').text.strip()
			citations_per_year[year_value] = int(citation_count)
			logging.info(f'Citation data for article ID {article_id}: , year: {year_value}, num of citation: {citations_per_year}')
		
		return citations_per_year
	except Exception as e:
		print(f'Error accessing {url}: {e}')
		return {}

browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# DataFrame to store citation information
citation_infos_df = pd.DataFrame(columns=['article_id', 'year', 'citations'])

# Process each article
for index, row in results_df.iterrows():
	article_id = row['article_id']
	article_url = row['article_url']
	logging.info(f"Accessing article ID {article_id} with URL {article_url}")

	citations_per_year = get_citation_data(article_id, article_url)

	# Insert citation data into DataFrame
	for year, citations in citations_per_year.items():
		citation_infos_df = pd.concat([
			citation_infos_df,
			pd.DataFrame({'article_id': [article_id], 'year': [year], 'citations': [citations]})
		], ignore_index=True)
		logging.info(f"Inserting citation data for article ID {article_id}: year: {year}, num of citation: {citations}")

# Close the browser
browser.quit()

# Display the DataFrame
print(citation_infos_df.head())

2025-01-21 15:36:38,004 - INFO - ====== WebDriver manager ======
2025-01-21 15:36:38,187 - INFO - Get LATEST chromedriver version for google-chrome
2025-01-21 15:36:39,216 - INFO - Get LATEST chromedriver version for google-chrome
2025-01-21 15:36:40,035 - INFO - Driver [/Users/rudyhendrawan/.wdm/drivers/chromedriver/mac64/131.0.6778.264/chromedriver-mac-arm64/chromedriver] found in cache
2025-01-21 15:36:40,885 - INFO - Accessing article ID qjU7YEgAAAAJ:ufrVoPGSRksC with URL /citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:ufrVoPGSRksC
2025-01-21 15:36:45,282 - INFO - Getting citation data for article ID qjU7YEgAAAAJ:ufrVoPGSRksC - URL: /citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:ufrVoPGSRksC
2025-01-21 15:36:45,293 - INFO - Citation data for article ID qjU7YEgAAAAJ:ufrVoPGSRksC: , year: 2021, num of citation: {'2021': 6}
2025-01-21 15:36:45,294 - INFO - Citation data for article ID qjU7YEgAAAAJ:ufrVoPG

No graph bars found for https://scholar.google.com/citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:4TOpqqG69KYC


2025-01-21 15:37:41,819 - INFO - Getting citation data for article ID qjU7YEgAAAAJ:_kc_bZDykSQC - URL: /citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:_kc_bZDykSQC
2025-01-21 15:37:41,838 - INFO - Accessing article ID qjU7YEgAAAAJ:YOwf2qJgpHMC with URL /citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:YOwf2qJgpHMC


No graph bars found for https://scholar.google.com/citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:_kc_bZDykSQC


2025-01-21 15:37:44,202 - INFO - Getting citation data for article ID qjU7YEgAAAAJ:YOwf2qJgpHMC - URL: /citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:YOwf2qJgpHMC
2025-01-21 15:37:44,217 - INFO - Accessing article ID qjU7YEgAAAAJ:kNdYIx-mwKoC with URL /citations?view_op=view_citation&hl=en&user=qjU7YEgAAAAJ&citation_for_view=qjU7YEgAAAAJ:kNdYIx-mwKoC


...

No graph bars found for https://scholar.google.com/citations?view_op=view_citation&hl=en&user=Ztn650AAAAAJ&cstart=20&pagesize=80&citation_for_view=Ztn650AAAAAJ:ufrVoPGSRksC


2025-01-21 15:45:03,733 - INFO - Getting citation data for article ID Ztn650AAAAAJ:zYLM7Y9cAGgC - URL: /citations?view_op=view_citation&hl=en&user=Ztn650AAAAAJ&cstart=20&pagesize=80&citation_for_view=Ztn650AAAAAJ:zYLM7Y9cAGgC
2025-01-21 15:45:03,750 - INFO - Accessing article ID Ztn650AAAAAJ:IjCSPb-OGe4C with URL /citations?view_op=view_citation&hl=en&user=Ztn650AAAAAJ&cstart=20&pagesize=80&citation_for_view=Ztn650AAAAAJ:IjCSPb-OGe4C


No graph bars found for https://scholar.google.com/citations?view_op=view_citation&hl=en&user=Ztn650AAAAAJ&cstart=20&pagesize=80&citation_for_view=Ztn650AAAAAJ:zYLM7Y9cAGgC


2025-01-21 15:45:06,089 - INFO - Getting citation data for article ID Ztn650AAAAAJ:IjCSPb-OGe4C - URL: /citations?view_op=view_citation&hl=en&user=Ztn650AAAAAJ&cstart=20&pagesize=80&citation_for_view=Ztn650AAAAAJ:IjCSPb-OGe4C


No graph bars found for https://scholar.google.com/citations?view_op=view_citation&hl=en&user=Ztn650AAAAAJ&cstart=20&pagesize=80&citation_for_view=Ztn650AAAAAJ:IjCSPb-OGe4C
                  article_id  year citations
0  qjU7YEgAAAAJ:ufrVoPGSRksC  2021         6
1  qjU7YEgAAAAJ:ufrVoPGSRksC  2022        16
2  qjU7YEgAAAAJ:ufrVoPGSRksC  2023        18
3  qjU7YEgAAAAJ:ufrVoPGSRksC  2024        16
4  qjU7YEgAAAAJ:ULOm3_A8WrAC  2020         3


In [5]:
citation_infos_df.tail()

,article_id,year,citations
131,Ztn650AAAAAJ:roLk4NBRz8UC,2023,1
132,Ztn650AAAAAJ:WF5omc3nYNoC,2022,1
133,Ztn650AAAAAJ:W7OEmFMy1HYC,2025,1
134,Ztn650AAAAAJ:Y0pCki6q_DkC,2022,1
135,Ztn650AAAAAJ:d1gkVwhDpl0C,2023,1


In [6]:
# Merge citation_infos_df with results_df on article_id
merged_df = pd.merge(citation_infos_df, results_df, on='article_id', how='inner')

# Join the merged_df with df_samples on scholar_id
final_df = pd.merge(merged_df, df_samples, on='scholar_id', how='inner')

# Display the final DataFrame
final_df.head()

,article_id,year_x,citations_x,scholar_id,title,authors,publisher,year_y,citations_y,article_url,NO,NIDN,NAMA,NAMA DOSEN DENGAN GELAR,JAFUNG,SERDOS
0,qjU7YEgAAAAJ:ufrVoPGSRksC,2021,6,qjU7YEgAAAAJ,Konsep Algoritme dan Aplikasinya dalam Bahasa ...,"S Hanief, IW Jepriana","Penerbit Andi, 2020",2020,61,/citations?view_op=view_citation&hl=en&user=qj...,39,806079201.0,I WAYAN JEPRIANA,"I WAYAN JEPRIANA, S.Kom., M.Cs",Asisten Ahli,2024.0
1,qjU7YEgAAAAJ:ufrVoPGSRksC,2022,16,qjU7YEgAAAAJ,Konsep Algoritme dan Aplikasinya dalam Bahasa ...,"S Hanief, IW Jepriana","Penerbit Andi, 2020",2020,61,/citations?view_op=view_citation&hl=en&user=qj...,39,806079201.0,I WAYAN JEPRIANA,"I WAYAN JEPRIANA, S.Kom., M.Cs",Asisten Ahli,2024.0
2,qjU7YEgAAAAJ:ufrVoPGSRksC,2023,18,qjU7YEgAAAAJ,Konsep Algoritme dan Aplikasinya dalam Bahasa ...,"S Hanief, IW Jepriana","Penerbit Andi, 2020",2020,61,/citations?view_op=view_citation&hl=en&user=qj...,39,806079201.0,I WAYAN JEPRIANA,"I WAYAN JEPRIANA, S.Kom., M.Cs",Asisten Ahli,2024.0
3,qjU7YEgAAAAJ:ufrVoPGSRksC,2024,16,qjU7YEgAAAAJ,Konsep Algoritme dan Aplikasinya dalam Bahasa ...,"S Hanief, IW Jepriana","Penerbit Andi, 2020",2020,61,/citations?view_op=view_citation&hl=en&user=qj...,39,806079201.0,I WAYAN JEPRIANA,"I WAYAN JEPRIANA, S.Kom., M.Cs",Asisten Ahli,2024.0
4,qjU7YEgAAAAJ:ULOm3_A8WrAC,2020,3,qjU7YEgAAAAJ,Framework Itil V3 Domain Service Operation Dal...,"S Hanief, IW Jefriana","Jurnal Teknologi Informasi dan Komputer 4 (1),...",2018,47,/citations?view_op=view_citation&hl=en&user=qj...,39,806079201.0,I WAYAN JEPRIANA,"I WAYAN JEPRIANA, S.Kom., M.Cs",Asisten Ahli,2024.0
